In [1]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient
import logging
import time
import argparse
import json

In [2]:
%cd iot-beyoung-01/

/home/ec2-user/SageMaker/myAWSStudyBlog/AIoT-secure-upload/iot-beyoung-01


In [3]:
host = 'a2xqi9tklutvnu-ats.iot.us-west-2.amazonaws.com'
rootCAPath = 'root-CA.crt'
certificatePath = 'iot-beyoung-01.cert.pem'
privateKeyPath = 'iot-beyoung-01.private.key'
port = 8883
clientId = 'basicPubSub'
topic = 'sdk/test/Python'

In [ ]:
# # Configure logging
# logger = logging.getLogger("AWSIoTPythonSDK.core")
# logger.setLevel(logging.DEBUG)
# streamHandler = logging.StreamHandler()
# formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
# streamHandler.setFormatter(formatter)
# logger.addHandler(streamHandler)

In [4]:
myAWSIoTMQTTClient = AWSIoTMQTTClient(clientId)
myAWSIoTMQTTClient.configureEndpoint(host, port)
myAWSIoTMQTTClient.configureCredentials(rootCAPath, privateKeyPath, certificatePath)

In [5]:
myAWSIoTMQTTClient.configureAutoReconnectBackoffTime(1, 32, 20)
myAWSIoTMQTTClient.configureOfflinePublishQueueing(-1)  # Infinite offline Publish queueing
myAWSIoTMQTTClient.configureDrainingFrequency(2)  # Draining: 2 Hz
myAWSIoTMQTTClient.configureConnectDisconnectTimeout(10)  # 10 sec
myAWSIoTMQTTClient.configureMQTTOperationTimeout(5)  # 5 sec

In [6]:
# Connect and subscribe to AWS IoT
myAWSIoTMQTTClient.connect()

True

In [7]:
message = {}
message['message'] = "hello from the jupyter"
message['sequence'] = 2
messageJson = json.dumps(message)
myAWSIoTMQTTClient.publish(topic, messageJson, 1)

True

# You can see the result on AWS MQTT Test Client

![](./images/08.png)

# Change the device policy

We need to change the device policy. So that we can post the topic to 'iot/presigned_upload'. 

In AWS IoT > Security > Policies > iot-beyoung-01-Policy.

* Add topics topic/iot/presigned_upload and topic/iot/presigned_upload/result
* Add client id as iot-beyoung-nb

**important**

* client id will be checked when you send the topic

```
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "iot:Publish",
        "iot:Receive"
      ],
      "Resource": [
        "arn:aws:iot:us-west-2:476271697919:topic/sdk/test/Python",
        "arn:aws:iot:us-west-2:476271697919:topic/iot/presigned_upload/result",
        "arn:aws:iot:us-west-2:476271697919:topic/iot/presigned_upload"
      ]
    },
    {
      "Effect": "Allow",
      "Action": [
        "iot:Subscribe"
      ],
      "Resource": [
        "arn:aws:iot:us-west-2:476271697919:topicfilter/sdk/test/Python",
        "arn:aws:iot:us-west-2:476271697919:topicfilter/iot/presigned_upload/result"
      ]
    },
    {
      "Effect": "Allow",
      "Action": [
        "iot:Connect"
      ],
      "Resource": [
        "arn:aws:iot:us-west-2:476271697919:client/basicPubSub",
        "arn:aws:iot:us-west-2:476271697919:client/presignedURL"
      ]
    }
  ]
}
```

## Send message to topic presigned upload

* Subscribe the 'iot/presigned_upload/result' in the AWS IoT console
* Run the following script

In [8]:
topic = 'iot/presigned_upload'
message = {
  "bucket": "beyoung-pb-upload-download",
  "prefix": "iot-devices",
  "name": "bear-01.jpg",
  "expiration": 300
}

messageJson = json.dumps(message)
myAWSIoTMQTTClient.publish(topic, messageJson, 1)

True

![](./images/09.png)